In [50]:
import pandas as pd

In [51]:
df1 = pd.read_csv('model1.csv', header=None, index_col=0)
df2 = pd.read_csv('model2.csv', header=None, index_col=0)

In [52]:
df1.index = df1.index.fillna("model")
df2.index = df2.index.fillna("model")

In [53]:
df1

,1,2
0,,
model,Alicia 1013,Alicia 1013
AAA,1,7
BBB,2,8
CCC,3,9
DDD,4,10
EEE,5,11
FFF,6,12


In [54]:
df2

,1,2
0,,
model,Alicia 1013A,Alicia 1013A
AAA,1,7
AAA1,2,8
BBB,3,9
BBB1,4,10
GGG,5,11
HHH,6,12


In [55]:
df_merge = pd.concat([df2,df1],axis=1)

In [56]:
df_merge

,1,2,1,2
0,,,,
model,Alicia 1013A,Alicia 1013A,Alicia 1013,Alicia 1013
AAA,1,7,1,7
AAA1,2,8,NaN,NaN
BBB,3,9,2,8
BBB1,4,10,NaN,NaN
GGG,5,11,NaN,NaN
HHH,6,12,NaN,NaN
CCC,NaN,NaN,3,9
DDD,NaN,NaN,4,10


In [57]:
df_merge = df_merge.T

In [58]:
df_merge

,model,AAA,AAA1,BBB,BBB1,GGG,HHH,CCC,DDD,EEE,FFF
1,Alicia 1013A,1,2,3,4,5,6,NaN,NaN,NaN,NaN
2,Alicia 1013A,7,8,9,10,11,12,NaN,NaN,NaN,NaN
1,Alicia 1013,1,NaN,2,NaN,NaN,NaN,3,4,5,6
2,Alicia 1013,7,NaN,8,NaN,NaN,NaN,9,10,11,12


In [59]:
df_merge.to_csv("出力ファイル名.csv", index=False, encoding="utf-8")

In [60]:
import pandas as pd
import os

# フォルダ内のCSVファイルを処理する関数
def process_csv_files(folder_path):
    df_list = []  # データフレームを保存するリスト

    # フォルダ内のすべてのCSVファイルを取得
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.csv'):
            file_path = os.path.join(folder_path, file_name)
            
            # CSVを読み込んで処理
            df = pd.read_csv(file_path, header=None, index_col=0)
            df.index = df.index.fillna("model")
            df_list.append(df)
    
    # 全てのデータフレームを連結
    df_merge = pd.concat(df_list,axis=1)
    return df_merge

# フォルダのパスを指定して関数を実行
folder_path = 'test'  # フォルダのパスを入力
result_df = process_csv_files(folder_path)
result_df = result_df.T

# 結果を確認
result_df.to_csv("出力ファイル名.csv", index=False, encoding="utf-8")


In [61]:
result_df

,model,AAA,AAA1,BBB,BBB1,GGG,HHH,CCC,DDD,EEE,FFF
1,Alicia 1013A,1,2,3,4,5,6,NaN,NaN,NaN,NaN
2,Alicia 1013A,7,8,9,10,11,12,NaN,NaN,NaN,NaN
1,Alicia 1013,1,NaN,2,NaN,NaN,NaN,3,4,5,6
2,Alicia 1013,7,NaN,8,NaN,NaN,NaN,9,10,11,12
